# Custom RAG Pipeline w/ History Implementation

This program will be adding history to our RAG pipeline, modifying history from built in RAG history retrievers to creating custom chains with the "|" operator. Creating a custom retriever allows us better control over context injection, injecting in smaller chunks with metadata chunks.

In [1]:
import os
import faiss
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaLLM

In [2]:
MODEL_NAME = "llama3.2"
llm = OllamaLLM(model= MODEL_NAME)

In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_id = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

tokenizer.save_pretrained("local_models/flan-t5-small")
model.save_pretrained("local_models/flan-t5-small")


/home/Akshay/miniconda3/envs/LLM_Train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def load_docs():
    
    document_loader = []

    for root, dirs, files in os.walk("."):
        # Skip chroma_db folder
        if "faiss" in root or "git" in root:
            continue
        for file in files:
            if file.endswith(".pdf"):
                document_loader.append(file)

    return document_loader

In [4]:
document_loader = load_docs()
document_loader

['ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf',
 'Three Phase Power System Fundamentals.pdf',
 'ENSC3016_Course_Notes_Part_2_Electric_Machines.pdf',
 'Electric Machinery Fundamentals Textbook -- Chapman.pdf',
 'ENSC3016 Study Guide 1-Review of Circuit Fundamentals.pdf']

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
model.save("./local_models/all-MiniLM-L6-v2")

In [5]:
embedding_model ="./local_models/all-MiniLM-L6-v2" #embedding matrix model

def embed_splitting(document_loader, embedding_model):
    embeddings = HuggingFaceEmbeddings(model = embedding_model, encode_kwargs={'normalize_embeddings': True})

    doc_store = []
    for file in document_loader:
        loader = PyPDFLoader(file)
        doc = loader.load()
        doc_store += doc

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size = 400,
        chunk_overlap = 64
        )
    
    #Make splits
    splits = text_splitter.split_documents(doc_store)

    return embeddings, splits

In [6]:
embeddings, splits = embed_splitting(document_loader, embedding_model)

/home/Akshay/miniconda3/envs/LLM_Train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
example_split = splits[106]
example_split

Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 51, 'page_label': '52'}, page_content='Transformer 52 \n \n \n \n   Figure 6-3 Shell-type transformers. \n \n \n \nFigure 6-4 Flux plot: shell-type transformer \n \n \nToroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. \nAlthough they are more expensive than shell-type transformers, the performance is better. They are used \nin high -quality electronic equipment and for instrument transformers (see section 6.3) where \nmeasurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. \n \nFigure 6-5 Toroidal transformers.\uf020\n \n \n \n6.2 Transformer Principle: \nThe action of a transformer is most easily underst

In [8]:
metadata = example_split.metadata
for key in metadata:
    print(f"{key}: {metadata[key]}")

producer: Microsoft® Word 2013
creator: Microsoft® Word 2013
creationdate: 2019-07-27T15:04:48+08:00
author: Ali Kharrazi
moddate: 2019-07-27T15:04:48+08:00
source: ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf
total_pages: 76
page: 51
page_label: 52


In [9]:
embeddings

HuggingFaceEmbeddings(model_name='./local_models/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={'normalize_embeddings': True}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [10]:
len(splits)

402

In [11]:
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [12]:
dim = len(embeddings.embed_query("test sentence"))
index = faiss.IndexFlatL2(dim)

if os.path.exists("faiss_index"):
    print("Loading FAISS index from disk...")
    vector_store = FAISS.load_local("faiss_index", embeddings=embeddings, allow_dangerous_deserialization=True)
else:
    print("Building FAISS index from scratch...")
    vector_store = FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
    )
    vector_store.add_documents(splits)
    vector_store.save_local("faiss_index")

Loading FAISS index from disk...


In [13]:
# create the retriever object once
semantic_retriever = vector_store.as_retriever(search_kwargs={'k': 4})

# define your function to query it
def semantic_search(retriever_obj, input_context: str):
    return retriever_obj.invoke(input_context)

# call the function with retriever and query string
results = semantic_search(semantic_retriever, "Explain transformers")

In [14]:
results[0].metadata

{'producer': 'Microsoft® Word 2013',
 'creator': 'Microsoft® Word 2013',
 'creationdate': '2019-07-27T15:04:48+08:00',
 'author': 'Ali Kharrazi',
 'moddate': '2019-07-27T15:04:48+08:00',
 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf',
 'total_pages': 76,
 'page': 51,
 'page_label': '52'}

In [15]:
for i in range(len(results)):
    source_data = results[i].metadata["source"]
    page = results[i].metadata["page"]
    page_content = results[i].page_content

    print(f"This is chunk number {i+1}.\n\n The source is {source_data}, found on page number {page}. \n\n The page content is {page_content} \n")

This is chunk number 1.

 The source is ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, found on page number 51. 

 The page content is Transformer 52 
 
 
 
   Figure 6-3 Shell-type transformers. 
 
 
 
Figure 6-4 Flux plot: shell-type transformer 
 
 
Toroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. 
Although they are more expensive than shell-type transformers, the performance is better. They are used 
in high -quality electronic equipment and for instrument transformers (see section 6.3) where 
measurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. 
 
Figure 6-5 Toroidal transformers.
 
 
 
6.2 Transformer Principle: 
The action of a transformer is most easily understood if the two coils are wound on opposite sides of a 
magnetic core, as shown in the model of figure 6 -6. This form is used for some low -cost transformers, 
but the magnetic coupling is not as good as with the shel

In [16]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(splits)
bm25_retriever.k = 4

def keyword_search(retriever_obj, input_context: str):
    return retriever_obj.invoke(input_context)

In [17]:
keyword_results = keyword_search(bm25_retriever, "Explain transformers")

In [18]:
keyword_results

[Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 64, 'page_label': '65'}, page_content='65 Electrical Machines and Systems                                                                                                            \n6.8 Current Transformers \nInstrument transformers are special transformers for extending the range of measur ing instruments. \nThere are two basic types: voltage transformers for measuring high voltages, and current transformers \nfor measuring high currents. Using transformers for voltage measurement is similar in principle to the \nordinary use of transformers to ch ange voltage levels, so it will not be considered further. Current \ntransformers, on the other hand, need special consideration. These are

In [19]:
for i in range(len(keyword_results)):
    source_data = keyword_results[i].metadata["source"]
    page = keyword_results[i].metadata["page"]
    page_content = keyword_results[i].page_content

    print(f"This is chunk number {i+1}.\n\n The source is {source_data}, found on page number {page}. \n\n The page content is {page_content} \n")

This is chunk number 1.

 The source is ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, found on page number 64. 

 The page content is 65 Electrical Machines and Systems                                                                                                            
6.8 Current Transformers 
Instrument transformers are special transformers for extending the range of measur ing instruments. 
There are two basic types: voltage transformers for measuring high voltages, and current transformers 
for measuring high currents. Using transformers for voltage measurement is similar in principle to the 
ordinary use of transformers to ch ange voltage levels, so it will not be considered further. Current 
transformers, on the other hand, need special consideration. These are usually toroidal transformers with 
high-quality core material. 
Figure 6-25 shows a load connected to a source. The primary of a current transformer is in series with 
the load, and the secondary 

In [20]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(retrievers= [semantic_retriever, bm25_retriever], weights = [0.5, 0.5])

In [21]:
combined_results = ensemble_retriever.invoke("Explain transformers")

In [22]:
combined_results

[Document(id='402198b9-0143-476b-beba-33c8d40d298b', metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 51, 'page_label': '52'}, page_content='Transformer 52 \n \n \n \n   Figure 6-3 Shell-type transformers. \n \n \n \nFigure 6-4 Flux plot: shell-type transformer \n \n \nToroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. \nAlthough they are more expensive than shell-type transformers, the performance is better. They are used \nin high -quality electronic equipment and for instrument transformers (see section 6.3) where \nmeasurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. \n \nFigure 6-5 Toroidal transformers.\uf020\n \n \n \n6.2 Transformer Principle: \nThe ac

In [23]:
for i in combined_results:

    print(i.metadata)

{'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 51, 'page_label': '52'}
{'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 64, 'page_label': '65'}
{'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 50, 'page_label': '51'}
{'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013'

In [24]:
i = 1
result_list = [combined_results[0]]
seen_pages = {combined_results[0].metadata["page_label"]}

while i < len(combined_results):
    metadata = combined_results[i].metadata
    page_label = metadata["page_label"]

    if page_label in seen_pages:
        i += 1  # You MUST increment i here
        continue

    result_list.append(combined_results[i])
    seen_pages.add(page_label)
    i += 1

In [25]:
result_list

[Document(id='402198b9-0143-476b-beba-33c8d40d298b', metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2019-07-27T15:04:48+08:00', 'author': 'Ali Kharrazi', 'moddate': '2019-07-27T15:04:48+08:00', 'source': 'ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf', 'total_pages': 76, 'page': 51, 'page_label': '52'}, page_content='Transformer 52 \n \n \n \n   Figure 6-3 Shell-type transformers. \n \n \n \nFigure 6-4 Flux plot: shell-type transformer \n \n \nToroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. \nAlthough they are more expensive than shell-type transformers, the performance is better. They are used \nin high -quality electronic equipment and for instrument transformers (see section 6.3) where \nmeasurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. \n \nFigure 6-5 Toroidal transformers.\uf020\n \n \n \n6.2 Transformer Principle: \nThe ac

In [26]:
input_template = """You are an expert assistant answering based only on the provided context.

The retrieved documents have been joined together and are separated by "Chunk_n", where n is the chunk number. Here is the context:

{context}

Use all relevant information above to answer the question below. If the answer isn't found in the chunks, say:
"I cannot answer this question because the necessary information was not found in the provided documents."

❗Do not cite or mention any source files or page numbers in the body of your answer.

At the end of your answer, add a single line in this format:

Information was pulled from: <source_file_1>: pages <comma-separated page numbers>; <source_file_2>: pages <...>; ...

Use only one entry per document, listing all unique page numbers where information was pulled from.
Do not mention metadata_n, chunk_n, or include references in the main answer.

Metadata:
{metadata}

Question: {question}
"""

In [27]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnableParallel

def content_parser(results):
    context_string = ""
    for i in range(len(results)):

        context_string += f"\nChunk_{i+1}:\n\n{results[i].page_content}\n\n\n"

    return context_string 

chunks = content_parser(result_list)

In [28]:
chunk_runnable = RunnableLambda(content_parser)

In [29]:
def metadata_parser(results):
    metadata_files = {}

    for doc in results:
        source = doc.metadata["source"]
        page = doc.metadata["page_label"]

        if source in metadata_files:
            if page not in metadata_files[source]:
                metadata_files[source].append(page)
        else:
            metadata_files[source] = [page]

    metadata_string = "This file uses the following sources:"

    for key in metadata_files:
        pages = ", ".join(str(i) for i in metadata_files[key])
        metadata_string += f"\n\n{key}, pages {pages}"
        

    return metadata_string

In [30]:
metadata_runnable = RunnableLambda(metadata_parser)
print(metadata_runnable.invoke(result_list))

This file uses the following sources:

ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, pages 52, 65, 51, 53, 56, 69


In [31]:
metadata = metadata_parser(result_list)

In [32]:
print(metadata)

This file uses the following sources:

ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf, pages 52, 65, 51, 53, 56, 69


In [33]:
prompt_template = ChatPromptTemplate(
    [
        ("system", input_template),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt_template | llm

In [34]:
contextualize_q_system_prompt = """Given a chat history and the latest user question which might reference previous context,
reformulate it as a fully standalone question that can be answered without the chat history.
If the question is already standalone, return it as-is."""

contextual_prompt = ChatPromptTemplate(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


In [35]:
chain = prompt_template | llm
answer1 = chain.invoke(
    {'context': chunks,
     'metadata': metadata,
     'chat_history': [],
     'input': "Explain transformers",
     'question': "Explain transformers"}
)

In [36]:
answer1

'Transformers are practical applications of magnetically coupled coils used to transfer energy from one coil to another. The primary purpose of a transformer is to provide electrical isolation between the source and the load or to change the voltage and current levels.\n\nThe action of a transformer is based on the principle of magnetic coupling, where two coils are wound on opposite sides of a magnetic core. This form of construction improves the coupling and provides better performance compared to other types of transformers.\n\nIn a transformer, there are two main coils: the primary coil connected to the source and the secondary coil connected to the load. The ratio of turns in these coils determines the voltage transformation ratio, with the secondary voltage being either larger or smaller than the primary voltage by changing the ratio of turns on the two windings.\n\nTransformers can be used for various applications, including power transmission and distribution, electronic equipm

In [37]:
a = RunnableParallel({'context': chunk_runnable, 'metadata': metadata_runnable})
b = a.invoke(result_list)

In [38]:
b

{'context': '\nChunk_1:\n\nTransformer 52 \n \n \n \n   Figure 6-3 Shell-type transformers. \n \n \n \nFigure 6-4 Flux plot: shell-type transformer \n \n \nToroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. \nAlthough they are more expensive than shell-type transformers, the performance is better. They are used \nin high -quality electronic equipment and for instrument transformers (see section 6.3) where \nmeasurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. \n \nFigure 6-5 Toroidal transformers.\uf020\n \n \n \n6.2 Transformer Principle: \nThe action of a transformer is most easily understood if the two coils are wound on opposite sides of a \nmagnetic core, as shown in the model of figure 6 -6. This form is used for some low -cost transformers, \nbut the magnetic coupling is not as good as with the shell-type construction. \n \n \nFigure 6-6  Core-type transformer \n \n \n \nFigure 6 -7 is a schema

In [40]:
inputs = ({
    **b,
    'chat_history': [],
    'input': 'Explain transformers',
    'question': 'Explain transformers'
})

In [41]:
inputs

{'context': '\nChunk_1:\n\nTransformer 52 \n \n \n \n   Figure 6-3 Shell-type transformers. \n \n \n \nFigure 6-4 Flux plot: shell-type transformer \n \n \nToroidal transformers exploit the remarkable properties of toroidal coils described in section 3.6. \nAlthough they are more expensive than shell-type transformers, the performance is better. They are used \nin high -quality electronic equipment and for instrument transformers (see section 6.3) where \nmeasurement accuracy is important. Typical toroidal transformers are shown in figure 6-5. \n \nFigure 6-5 Toroidal transformers.\uf020\n \n \n \n6.2 Transformer Principle: \nThe action of a transformer is most easily understood if the two coils are wound on opposite sides of a \nmagnetic core, as shown in the model of figure 6 -6. This form is used for some low -cost transformers, \nbut the magnetic coupling is not as good as with the shell-type construction. \n \n \nFigure 6-6  Core-type transformer \n \n \n \nFigure 6 -7 is a schema

In [43]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [44]:
history_aware_chain = RunnableWithMessageHistory(
    chain,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

In [46]:
get_session_history('17egw2737').messages

[]

In [49]:
import uuid

def temp_pipeline():

    session_id = str(uuid.uuid4())[:8]
    print(f"Session ID: {session_id}")
    
    history = get_session_history(session_id)

    print(f"\nModel {MODEL_NAME} has been initiated with memory. Please feel free to ask questions or type 'exit' to quit.")
    while True:
        
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Session ended. Have a good day.")
            break

        print(f"{user_input}\n\n\n")
        
        recontextual_chain = contextual_prompt | llm
        rephrased_question = recontextual_chain.invoke(
            {'chat_history': history.messages,
             'input': user_input})
        
        print(f"{rephrased_question} \n\n\n")

        context_injection = (ensemble_retriever | RunnableParallel({'context': chunk_runnable, 'metadata': metadata_runnable})).invoke(rephrased_question)

        print("Context:\n", context_injection['context'])  # Preview the first 500 chars
        print("Metadata:\n", context_injection['metadata'])
        
        response = history_aware_chain.invoke(
            {**context_injection,
            'input': user_input,
            'question': rephrased_question},
            config={"configurable": {"session_id": session_id}}
        )
        
        print(f"LLM: {response}\n")

In [50]:
temp_pipeline()

Session ID: ba74ef42

Model llama3.2 has been initiated with memory. Please feel free to ask questions or type 'exit' to quit.
Explain transformers


"
Here's a reformulated version of your question:

"What are transformers in natural language processing?"

The original question "Human: Explain transformers" doesn't contain any reference to a specific context or previous discussion. Therefore, I returned it as-is since it can be answered without referencing the chat history. 


"
Context:
 
Chunk_1:

57 Electrical Machines and Systems                                                                                                            
The corresponding fluxes are shown in figure 6-12 
 
 
Figure6-12   3-phase transformer flux 
 
 
In a balanced system with sinusoidal phase voltages, the fluxes will be given by: 
𝜙𝑎 = 𝜙𝑚𝑐𝑜𝑠𝜔𝑡                                               
𝜙𝑏 = 𝜙𝑚 cos(𝜔𝑡 − 120°)       (6 − 24) 
𝜙𝑐 = 𝜙𝑚 cos(𝜔𝑡 − 240°)                             
Figure 6-13 shows f

In [ ]:
recontextualiser_chain = contextual_prompt | llm
recontextualizer = RunnableWithMessageHistory(recontextualiser_chain, get_session_history)

In [ ]:
type(recontextualizer)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

rephrase_chain = contextual_prompt | llm | StrOutputParser()

In [ ]:
rephrase_chain.invoke(
    {'chat_history': [answer1],
     'input': "What materials are they made of?"}
)

In [ ]:
type(answer1)

In [51]:
get_session_history('ba74ef42').messages

[HumanMessage(content='Explain transformers', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Transformers are a type of artificial neural network (ANN) used in natural language processing (NLP) for machine translation, text summarization, and other NLP tasks. They were introduced in 2017 by Vaswani et al. and have since become one of the most widely used architectures in NLP.\n\nIn a transformer, the input sequence is split into multiple sub-sequences or partitions, and each partition is processed independently using self-attention mechanisms. The self-attention mechanism allows the model to weigh the importance of different words in the input sequence relative to each other, rather than relying on traditional recurrent neural network (RNN) architectures that process the sequence one word at a time.\n\nThe transformer architecture consists of an encoder and a decoder. The encoder takes in the input sequence and generates a sequence of representations, which are then f